In [1]:
#!/usr/bin/env python3
"""
plot_sr_plus_energy_levels.py

Load Sr⁺ energy‐level data, optionally apply a Zeeman split,
configure layout and style, and plot the level diagram with transitions.
"""

import json
from pathlib import Path
from week3.models import Level
from week3.io import load_ion_data
from week3.splitters import ZeemanSplitter
from week3.layout import LayoutConfig
from week3.style import StyleConfig
from week3.plotter import plot_energy_levels

def main():
    # ─── 1) Load the raw JSON file into our data structure ────────────────────
    # The JSON must contain keys: "ion", "unit", "levels", and optional "transitions".
    # Each level entry is passed through Level(**entry) to create Level objects.
    data_path = Path("week3\SrStatePrep.json")
    raw = json.loads(data_path.read_text())
    levels = [Level(**entry) for entry in raw["levels"]]
    data = {
        "ion":         raw.get("ion", ""),
        "unit":        raw.get("unit", "cm⁻¹"),
        "levels":      levels,
        "transitions": raw.get("transitions", []),
        "title":       raw.get("title", ""),
    }

    # ─── 2) (Optional) Apply a small Zeeman splitting to the levels ───────────
    # This will take each Level and produce sub‐levels with m‐quantum ticks.
    zs = ZeemanSplitter(B=0.01)  # magnetic field strength in Tesla
    split_levels = []
    for lvl in data["levels"]:
        # keep the original level...
        split_levels.append(lvl)
        # ...and extend with its Zeeman‐split children
        split_levels.extend(zs.split(lvl))
    data["levels"] = split_levels

    # ─── 3) Build the layout configuration ────────────────────────────────────
    # - column_letters & positions determine horizontal grouping (S, P, D, F → columns)
    # - spacing: base horizontal distance between columns
    # - bar_half: half‐width of each energy bar
    # - x_jitter: maximum horizontal jitter for sub‐levels around parent
    # - y_jitter: base vertical jitter for stacking levels
    # - energy_group_key: function to bucket levels into fine‐structure groups
    # - energy_group_y_scale: multiplier for vertical fan of base-level groups
    # - sublevel_y_scale: multiplier for extra vertical separation of sub‐levels
    layout = LayoutConfig(
        column_letters        = ["S", "P", "D", "F"],
        column_positions      = [0, 0, 1, 3],
        spacing               = 1,
        bar_half              = 0.5,
        x_jitter              = 0.3,
        y_jitter              = 50.0,
        energy_group_key      = lambda lvl: int(lvl.energy // 10000),
        energy_group_y_scale  = 30,
        sublevel_uniform_spacing = 1000.0,   # distance between adjacent sublevels
        sublevel_uniform_centered = True   # whether to center around parent
    )

    # ─── 4) Build the style configuration ─────────────────────────────────────
    # StyleConfig defines colors, line widths, tick sizes, etc.
    style = StyleConfig()

    # ─── 5) Plot the diagram ──────────────────────────────────────────────────
    # - data: contains ion name, unit, levels, transitions
    # - layout: controls bar/tick placement
    # - style: controls colors and line widths
    # - show_axis=True draws the y‐axis; set to False to hide it
    plot_energy_levels(data, layout, style, show_axis=False)

if __name__ == "__main__":
    main()


ValueError: could not convert string to float: '-1/2'

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from week3.io        import load_ion_data
from week3.splitters import ZeemanSplitter

data = load_ion_data("week3/test.json")
levels = data["levels"]
zs     = ZeemanSplitter(B=0.01)

all_lvls = []
for lvl in levels:
    all_lvls.append(lvl)
    all_lvls.extend(zs.split(lvl))

for lvl in all_lvls:
    print(f"sublevel={lvl.sublevel} | {lvl.label} | E={lvl.energy:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: 'week3\\test.json'

In [ ]:
for lvl in all_lvls:
    print(lvl)

In [ ]:
from week3.io        import load_ion_data
from week3.splitters import ZeemanSplitter
from week3.style     import StyleConfig
from week3.layout    import LayoutConfig
from week3.plotter   import plot_energy_levels

# 1) load + split
data = load_ion_data("week3/test.json")
levels0 = data["levels"]
zs      = ZeemanSplitter(B=0.01)

all_lvls = []
for lvl in levels0:
    all_lvls.append(lvl)
    all_lvls.extend(zs.split(lvl))

# 2) configs – note we now supply both letters and integer positions
layout = LayoutConfig(
    column_letters   = ["S", "P", "D", "F"],    # the term‐symbol letters
    column_positions = [1,    2,    3,    4],   # the integer column each letter maps to
    spacing          = 0.5,
    bar_half         = 0.1,
    x_jitter         = 0.2,
    y_jitter         = 20000.0
)

from week3.layout import compute_x_map, compute_y_map
style = StyleConfig()
# after splitting
x_map = compute_x_map(all_lvls, layout, style)
y_map = compute_y_map(all_lvls, layout)
print("x_map=", x_map)
print("y_map=", y_map)
from week3.plotter import draw_levels, draw_transitions
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8,5))
draw_levels(ax, all_lvls, x_map, y_map, layout, style)
draw_transitions(ax, [], x_map, y_map)
# …


style = StyleConfig()  # your default colors, line‐widths, etc.

# 3) plot
data = {
    "ion":        "88Sr+",
    "levels":     all_lvls,
    "transitions":[]
}

plot_energy_levels(data, layout, style)
